In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import stan
import arviz as az

import nest_asyncio
nest_asyncio.apply()

In [7]:
N_old = 10
N_new = 10
h = 9
fa = 1

In [8]:
#Parameters: d (detect old), g (old guess)
one_high_threshold_mpt = """
data {
  int<lower=0> N_old;
  int<lower=0> N_new;
  int<lower=0> h;      // hits
  int<lower=0> fa;     // false alarms
}
parameters {
  real<lower=0, upper=1> d;   // probability of detecting an old item
  real<lower=0, upper=1> g;   // probability of guessing old
}
model {
  h  ~ binomial(N_old, d + (1 - d) * g);
  fa ~ binomial(N_new, (1 - d) * g);
}
"""

In [9]:
#Parameters: d_o (detect old), d_n (detect new), g (old guess)
two_high_threshold_mpt = """
data {
  int<lower=0> N_old;
  int<lower=0> N_new;
  int<lower=0> h;
  int<lower=0> fa;
}
parameters {
  real<lower=0, upper=1> d_o;  // P(detect old when old)
  real<lower=0, upper=1> d_n;  // P(detect old when new)
  real<lower=0, upper=1> g;   // P(guess old)
}
model {
  h  ~ binomial(N_old, d_o + (1 - d_o) * g);
  fa ~ binomial(N_new, (1 - d_n) * g);
}
"""

In [10]:
data = {"N_old": N_old, "N_new": N_new, "h": h, "fa": fa}

# fit one high threshold model
one_ht_model = stan.build(one_high_threshold_mpt, data=data)
fit_1ht = one_ht_model.sample(num_samples=2000)

# fit two high threshold model
two_ht_model = stan.build(two_high_threshold_mpt, data=data)
fit_2ht = two_ht_model.sample(num_samples=2000)

Building...

In file included from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math/prim/prob/std_normal_ccdf_log.hpp:5,
                 from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math/prim/prob.hpp:331,
                 from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math/prim.hpp:16,
                 from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math/rev.hpp:16,
                 from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math.hpp:19,
                 from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /home/schild/.cache/httpstan/4.13.0/models/ov366x2h/model_ov366x2h.cpp:2:
/home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math/prim


Building: 12.8s, done.Messages from stanc:
Sampling:   0%
Sampling:  25% (3000/12000)
Sampling:  50% (6000/12000)
Sampling:  75% (9000/12000)
Sampling: 100% (12000/12000)
Sampling: 100% (12000/12000), done.
Messages received during sampling:
  Gradient evaluation took 1.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.11 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.18 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.15 seconds.
  Adjust your expectations accordingly!


Building...

In file included from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math/prim/prob/std_normal_ccdf_log.hpp:5,
                 from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math/prim/prob.hpp:331,
                 from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math/prim.hpp:16,
                 from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math/rev.hpp:16,
                 from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math.hpp:19,
                 from /home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /home/schild/.cache/httpstan/4.13.0/models/btxso2hr/model_btxso2hr.cpp:2:
/home/schild/anaconda3/envs/cognitive/lib/python3.12/site-packages/httpstan/include/stan/math/prim


Building: 12.2s, done.Messages from stanc:
Sampling:   0%
Sampling:  25% (3000/12000)
Sampling:  50% (6000/12000)
Sampling:  75% (9000/12000)
Sampling: 100% (12000/12000)
Sampling: 100% (12000/12000), done.
Messages received during sampling:
  Gradient evaluation took 3e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.03 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.03 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.02 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.02 seconds.
  Adjust your expectations accordingly!


In [11]:
print("One High Threshold:")
print(az.summary(fit_1ht, var_names=["d", "g"]))
print("Two High Threshold")
print(az.summary(fit_2ht, var_names=["d_o", "d_n", "g"]))


One High Threshold:
    mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
d  0.738  0.135   0.470    0.948      0.002    0.001    6580.0    4882.0   
g  0.498  0.213   0.126    0.893      0.003    0.002    6355.0    4915.0   

   r_hat  
d    1.0  
g    1.0  
Two High Threshold
      mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
d_o  0.645  0.245   0.160    0.989      0.004    0.003    3304.0    3154.0   
d_n  0.634  0.246   0.145    0.985      0.005    0.003    3206.0    2852.0   
g    0.492  0.259   0.071    0.923      0.005    0.003    3014.0    4600.0   

     r_hat  
d_o    1.0  
d_n    1.0  
g      1.0  


The two models suggest slightly different key parameter values, with the one high threshold having an average detection of 0.738 and an average guess of 0.498 and the two high threshold having an average old detection of 0.645 and average guess of 0.492. While the guess averages are roughly the same, the (old) detection averages have a more substantial difference. This is likely due to the high accuracy of the one individual who took the recognition task and the small sample size.